# HW4 : A visit to the movie zoo!

![](https://vignette.wikia.nocookie.net/bojackhorseman/images/f/f2/HSACWDTK%3FDTKT%3FLFO%21%21.png/revision/latest?cb=20150720050503)

## Task
In this homework, your task is to visualize THREE non-typical charts on anything related to your favorite **movie star!**
This means you CANNOT use the Big 4 chart types or their close variants (i.e. Pie, Bar, Line and Scatter, Area, etc.)

You are free to use any other chart type whether or not they were covered in class!
The lecture on Visit To The Zoo is a good place to start to get ideas on what kinds of charts exist.

For the data, you are free to use any data source you deem fit.
For charting, we will NOT be constraining the technology you use. 
You are free to produce the charts in any way you would like.

You will be judged on
* Creativity
* Presentation Quality
* Data Quality (Did your visualization reveal something interesting?)

For extra credit, you can make a fully interactive visualization.

## Ideas for Data Collection

Here, we show an example of how to collect data about Arnold Schwarzenegger!
Do note that this is just an example of the kind of data you can collect.
You are **NOT** constrained
* To the same movie star (you can pick your own!)
* To the same *kind* of data
* To the same data sources
* or to anything else!

This assignment gives you the power to do what you like!

In [2]:
from imdbpie import ImdbFacade
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import urllib.request
import re

#### Get data for Arnold Schwarzenegger

In [3]:
# Get an instance of IMDb class
imdb = ImdbFacade()

# Search for Arnold Schwarzenegger
people = imdb.search_for_name('Keanu Reeves')
print(people)

(NameSearchResult(imdb_id='nm0000206', name='Keanu Reeves'), NameSearchResult(imdb_id='nm2172419', name='Keanna Reeves'), NameSearchResult(imdb_id='nm4474752', name='Keane Reeves'), NameSearchResult(imdb_id='nm2023349', name='François Brun'))


In [9]:
# Fetch information about him
keanu = imdb.get_name(people[0].imdb_id)

# What information do I have about him?
print('\n'.join([x for x in dir(keanu) if not x.startswith('__')]))

bios
birth_place
date_of_birth
filmography
gender
image
imdb_id
name


In [5]:
# How many movies does he have?
print(len(keanu.filmography))

381


In [6]:
# Let's fetch some more information about a movie
movie = imdb.get_title(keanu.filmography[-1])

In [51]:
# What information can I get about this movie?
print('\n'.join([x for x in dir(movie) if not x.startswith('__')]))

certification
creators
credits
directors
episode
episodes
genres
image
imdb_id
plot_outline
rating
rating_count
release_date
releases
runtime
season
stars
title
type
writers
year


In [10]:
print(movie.imdb_id)

tt0090683


In [32]:
from tqdm import tqdm_notebook
from time import sleep

def get_url_with_retry(url):
    try: 
        return urllib.request.urlopen(url, timeout=30).read()
    except:
        return get_url_with_retry(url)
all_locations = []
for movie_title in tqdm_notebook(keanu.filmography):
    movie = None
    try:
        movie = imdb.get_title(movie_title)
    except:
        continue
    url = 'https://www.imdb.com/title/' + movie.imdb_id + '/locations'
    source = get_url_with_retry(url)
    soup = BeautifulSoup(source,'lxml')
    locations = soup.find_all('div', class_=['sodavote'])
    for location in locations:
#         print(location.find('a', {'itemprop': 'url'}).contents[0][:-1])
        all_locations.append(location.find('a', {'itemprop': 'url'}).contents[0][:-1])
#     print(locations.find_all('a', {'itemprop': 'url'}))
#     print(movie.imdb_id, locations)

In [35]:
# only keep locations with at least one comma for heatmap use with arcgis, geocode addresses for heatmap
from arcgis import GIS
from arcgis.geocoding import get_geocoders, batch_geocode
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
gis = GIS(username="kgzhou_UCSDOnline", verify_cert=False)
geocoder = get_geocoders(gis)[0]

Enter password: ········


In [40]:
detailed_locations = [location for location in all_locations if len(location.split(',')) > 1]
geocoded_locations = batch_geocode(detailed_locations)

In [49]:
geocoded_df = pd.DataFrame(geocoded_locations)
geocoded_df['coordinates'] = geocoded_df['location'].apply(lambda x: Point(x['x'], x['y']))
gdf = gpd.GeoDataFrame(geocoded_df, geometry='coordinates', crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
gdf.to_file('film_location.shp')

https://arcg.is/1aOS50

In [56]:
from collections import defaultdict
starring = defaultdict(int)
for movie_title in tqdm_notebook(keanu.filmography):
    movie = None
    try:
        movie = imdb.get_title(movie_title)
    except:
        continue
    for star in movie.stars:
        starring[star.name] += 1

In [64]:
starring_lst = [(starring[x], x) for x in starring if x != "Keanu Reeves"]
starring_lst.sort(key=lambda x: x[0], reverse=True)
starring_lst

[(16, 'Laurence Fishburne'),
 (10, 'Carrie-Anne Moss'),
 (8, 'Alex Winter'),
 (8, 'Sandra Bullock'),
 (5, 'Ana de Armas'),
 (5, 'John Gaeta'),
 (5, 'Bill Pope'),
 (4, 'Winona Ryder'),
 (4, 'Mary Alice'),
 (4, 'Paula Abdul'),
 (4, 'Richard Linklater'),
 (4, 'Lana Wachowski'),
 (4, 'Michael Ballhaus'),
 (4, 'Jessica Alba'),
 (4, 'Ben Affleck'),
 (4, 'Jennifer Aniston'),
 (4, 'Monica Bellucci'),
 (4, 'Joel Silver'),
 (4, 'Jada Pinkett Smith'),
 (3, 'Tom Hanks'),
 (3, 'Tim Allen'),
 (3, 'Halle Berry'),
 (3, 'Riccardo Scamarcio'),
 (3, 'Ruby Rose'),
 (3, 'Lorenza Izzo'),
 (3, 'Tiger Hu Chen'),
 (3, 'Illeana Douglas'),
 (3, 'Kathy Bates'),
 (3, 'Hugo Weaving'),
 (3, 'Charlize Theron'),
 (3, 'Jason Isaacs'),
 (3, 'Greg Germann'),
 (3, 'Adrien Brody'),
 (3, 'Dennis Hopper'),
 (3, 'William Sadler'),
 (3, 'Gary Oldman'),
 (3, 'George Carlin'),
 (3, 'Drew Barrymore'),
 (3, 'Francis Ford Coppola'),
 (3, 'Antonio Banderas'),
 (3, 'Helmut Bakaitis'),
 (3, 'Jaume Figueras'),
 (3, 'Chris Matheson'),
 

In [65]:
from collections import defaultdict
years = []
for movie_title in tqdm_notebook(keanu.filmography):
    movie = None
    try:
        movie = imdb.get_title(movie_title)
    except:
        continue
    years.append(movie.year)
years

[None,
 1970,
 2002,
 2020,
 2020,
 2020,
 2019,
 2019,
 2019,
 2019,
 2018,
 2018,
 2018,
 2016,
 2017,
 2017,
 2017,
 2017,
 2017,
 2016,
 2016,
 2016,
 2016,
 2016,
 2016,
 2016,
 2013,
 2015,
 2014,
 2013,
 2013,
 2013,
 2012,
 2010,
 2008,
 2009,
 2008,
 2008,
 2006,
 2006,
 2005,
 2005,
 2005,
 2005,
 2003,
 2003,
 2003,
 2003,
 2003,
 2003,
 2001,
 2001,
 2000,
 2000,
 2000,
 1999,
 1999,
 1999,
 1997,
 1997,
 1996,
 1996,
 1995,
 1995,
 1994,
 1993,
 1993,
 1993,
 1993,
 1992,
 1991,
 1991,
 1991,
 1991,
 1991,
 1990,
 1990,
 1990,
 1987,
 1989,
 1981,
 1989,
 1988,
 1988,
 1988,
 1988,
 1970,
 1986,
 1987,
 1970,
 1986,
 1986,
 1986,
 1986,
 1986,
 1954,
 1986,
 1986,
 1985,
 1985,
 1985,
 1985,
 2007,
 1981,
 None,
 None,
 1981,
 1994,
 1996,
 2013,
 2007,
 2017,
 2011,
 2004,
 2017,
 2007,
 2012,
 2014,
 2012,
 2014,
 2007,
 2000,
 2012,
 2007,
 2013,
 2010,
 2011,
 2009,
 1998,
 2009,
 2008,
 2003,
 2007,
 2007,
 2007,
 2007,
 2007,
 2005,
 1972,
 2002,
 2005,
 1997,
 2005,

In [66]:
years_dict = defaultdict(int)
for year in years:
    if year is not None:
        years_dict[year] += 1
years_dict

defaultdict(int,
            {1970: 3,
             2002: 9,
             2020: 4,
             2019: 5,
             2018: 8,
             2016: 16,
             2017: 18,
             2013: 12,
             2015: 8,
             2014: 19,
             2012: 7,
             2010: 9,
             2008: 8,
             2009: 9,
             2006: 12,
             2005: 16,
             2003: 27,
             2001: 15,
             2000: 12,
             1999: 8,
             1997: 8,
             1996: 13,
             1995: 7,
             1994: 8,
             1993: 9,
             1992: 6,
             1991: 9,
             1990: 5,
             1987: 3,
             1989: 2,
             1981: 4,
             1988: 5,
             1986: 9,
             1954: 1,
             1985: 4,
             2007: 15,
             2011: 7,
             2004: 18,
             1998: 3,
             1972: 1,
             1975: 1,
             1952: 1,
             1979: 1,
             1984: 2,
   

In [116]:
html = """
    <div style="background-color:#FFDDDD">
    <h2> Warning! </h2>
    <p> This code below is meant to be an example of what you can do. <br>
        It is not guaranteed to work always, and will need to be tweaked!
    </p>
    </div>
"""
display(HTML(html))

#### Box office numbers

In [117]:
# Let's experiment with Terminator
imdb_id = 'tt0088247'

# Fetch the box office numbers
base = 'https://www.boxofficemojo.com'
url = base + '/title/' + imdb_id
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')

table = soup('th', text=re.compile(r'Release Group'))[0].parent.parent
group = table.findAll('tr', recursive=False)[1].find('a').get('href')
url = base + group

# Get total earnings domestic and international
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
earnings = soup('h2', text=re.compile(r'Rollout'))[0].parent.parent.findAll('div')
domestic = earnings[1].find('span', {'class': 'money'}).get_text()
domestic_url = earnings[1].find('a').get('href')
international = earnings[2].find('span', {'class': 'money'}).get_text()

# Get weekly domestic earnings
url = base + domestic_url
url = url[:url.rfind('/')] + '/weekly/'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
table = soup.find('div', {'class':'a-section imdb-scroll-table-inner'}).findAll('tr')
weekly = []
for tr in table[1:]:
    date = tr.findAll('td')[0].get_text()
    earning = tr.findAll('td')[2].get_text()
    weekly.append((date, earning))

# Print the values we've just got!
print("Total Domestic Earnings: %s" % domestic)
print("Total International Earnings: %s" % international)
print("Weekly Domestic Earnings:")
for date, earning in weekly:
    print("\t%s \t: %s" % (date, earning))

Total Domestic Earnings: $38,371,200
Total International Earnings: $40,000,000
Weekly Domestic Earnings:
	Oct 26-Nov 1 	: $5,560,930
	Nov 2-8 	: $5,752,018
	Nov 9-15 	: $4,764,277
	Nov 16-22 	: $5,208,881
	Nov 23-29 	: $4,851,910
	Nov 30-Dec 6 	: $3,741,759
	Dec 7-13 	: $2,002,925
	Dec 14-20 	: $1,399,646
	Dec 21-27 	: $722,260
	Jan 11-17 	: $752,723
	Jan 18-24 	: $618,670


## Other resources!

This assignment doesn't have a restriction on where you can look for data.
Further, we don't mind how you collect the data, or what data you collect.

Here are some additional resources for this example, and you can customize it for your own!
* Arnold Schwarzenegger Kill Count: https://www.youtube.com/watch?v=OE6jpTaOYMU
* Arnold Schwarzenegger Top Quotes: https://www.youtube.com/watch?v=pDxn0Xfqkgw

You could think about the IMDB network as a graph, with different actors connected through movies.

Some other useful libraries/ databases:
* IMDBPy
* http://www.omdbapi.com/